In [1]:
import numpy as np
from scipy.stats import ortho_group
import sys
sys.path.append("../")
import numpy as np 
from utils import *
from lattice import *

### Define swap operator

In [2]:
bd = 2
S = np.zeros((bd,)*4)
for i in range(bd):
    for j in range(bd):
        S[i,j,j,i] = 1
S = S.reshape(bd**2, bd**2)

In [3]:
Es, Vs = np.linalg.eigh(S)

### Define unitary transformation with above symmetry 

In [4]:
U_tmp = np.zeros((bd**2,bd**2))
idx = np.argwhere(Es == -1)[-1]
U_tmp[:idx[0]+1, :idx[0]+1] = ortho_group.rvs(dim=np.round(idx[0]+1)) if idx[0] != 0 else 1
U_tmp[idx[0]+1:, idx[0]+1:] = ortho_group.rvs(dim=np.round(bd**2-idx[0]-1))
U = Vs @ U_tmp @ Vs.T

### Define transfer matrix with degenerated eigenvalues



In [9]:
lam_tmp = np.sort(np.random.uniform(low=-0.5,high = 1, size = (bd**2)))
lam_tmp[-2:] = 1
lam_tmp[0] = 0
lam = Vs @ np.diag(lam_tmp) @ Vs.T
A_transfer = U @ lam @ U.T
print("check if this is symmetric : ",np.linalg.norm(S @ A_transfer @ S - A_transfer))

check if this is symmetric :  5.551115123125783e-17


### Create MPS from transfer matrix

```
k - | A | - l
i - |  | - j
```

should be symmetric to $ i \leftrightarrow j, k \leftrightarrow l $ ( vertical flip) and horizonal flip 

### Return spectrum of transfer matrix

In [26]:
B = FF.block1D(bd, 3, bd, canonical=True).real
B_transfer = np.einsum("ipj, kpl->ikjl",B, B).reshape((bd**2,)*2)
print("spectrum of transfer matrix of canonical MPS : ", np.sort(np.linalg.eigvals(B_transfer)))

Ea:  [-0.19392068 -0.14106267  0.21596272  1.        ]
spectrum of transfer matrix of canonical MPS :  [-0.19392068 -0.14106267  0.21596272  1.        ]


In [24]:
A = A_transfer.reshape((bd,)*4)
print("confirm A has symmetry : ",np.linalg.norm(A.transpose((1,0,3,2))- A), np.linalg.norm(A.transpose((2,3,0,1))- A))
print("confirm spectrum is degenerated : ", np.round(np.linalg.eigvals(A.reshape((bd**2,)*2)),5))

confirm A has symmetry :  5.551115123125783e-17 1.1775693440128312e-16
confirm spectrum is degenerated :  [-0.48756  1.       1.       0.     ]


- Next, bundle $(k, l)$ and $(i, j)$. change the view of matrix
- Make sure B is symmetric matrix

In [20]:
B = np.einsum("ijkl->ikjl",A).reshape(bd**2, bd**2)
print(np.linalg.eigvalsh(B))

[-0.09130237  0.05516803  0.32703293  1.64722999]


In [249]:
E, V = np.linalg.eigh(B)
np.linalg.norm(V @ np.diag(E) @ V.T-B), E

(1.1951300559350904e-15,
 array([-0.73672021, -0.59216896,  0.91357297,  1.36804351]))

```
k - | A^T  A = B | - i
l - |            | - j
```

```
k - | db^T |  -  | db | - 
l - |      |     |    | -  
```

In [8]:
db = np.diag(np.sqrt(E)) @ V.T
np.linalg.norm(db.T @ db - B)

/var/folders/z7/m5qzz0hj12z06hxsn5km2c400000gn/T/ipykernel_41605/1316602744.py:1: RuntimeWarning: invalid value encountered in sqrt
  db = np.diag(np.sqrt(E)) @ V.T


nan

### invariant under reflective symmetry

```   
      py
       |
i - | da | -   j
```

In [9]:
A_MPS = db.reshape(4, 2, 2)
np.linalg.norm(A_MPS - A_MPS.transpose(0, 2, 1))

nan

### Reconstruct transition matrix

In [129]:
A_tilde_ = np.einsum("pij,pkl->ikjl", A_MPS, A_MPS)
A_tilde = A_tilde_.reshape(4,4)

In [130]:
np.linalg.norm(A_tilde - A0)

nan

In [131]:
A_tilde_ - A

array([[[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]],


       [[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]]])

In [27]:
import numpy as np